In [1]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex

import import_ipynb

import data_utils as data


tt = gp.Model('IIITB Exams Timetable')

973
59
[(15, 30), (7, 26), (40, 41), (48, 54), (11, 23), (41, 42), (10, 36), (10, 45), (2, 41), (10, 54), (3, 24), (3, 33), (3, 42), (14, 51), (36, 48), (47, 48), (7, 21), (17, 53), (9, 49), (29, 36), (48, 49), (6, 43), (29, 54), (10, 31), (10, 49), (22, 23), (22, 32), (14, 37), (55, 56), (5, 55), (29, 31), (2, 22), (25, 33), (22, 27), (3, 23), (13, 55), (16, 51), (28, 34), (17, 34), (36, 47), (9, 30), (9, 48), (21, 22), (6, 33), (10, 30), (2, 26), (24, 32), (35, 41), (14, 18), (16, 37), (17, 38), (28, 38), (21, 26), (39, 43), (2, 3), (24, 27), (3, 4), (51, 53), (1, 34), (16, 50), (5, 22), (9, 20), (9, 29), (9, 47), (6, 32), (2, 7), (20, 43), (12, 48), (4, 44), (16, 18), (1, 38), (35, 40), (5, 8), (17, 19), (38, 51), (17, 28), (5, 26), (5, 44), (27, 41), (50, 52), (8, 55), (20, 29), (39, 42), (23, 25), (23, 43), (4, 39), (1, 15), (24, 26), (32, 39), (5, 12), (15, 53), (9, 28), (8, 32), (42, 43), (11, 46), (12, 20), (12, 29), (4, 25), (12, 47), (4, 43), (1, 19), (34, 53), (1, 28), (5, 7

## Make one binary variable for each room-course-timeslot-day combination
$$
E_{crtd} =
\begin{cases} 
1 & \text{if a course $c$ exam is scheduled in room $r$ at time $t$ on day $d$,} \\
0 & \text{Otherwise.}
\end{cases}
$$

In [2]:
def exam_length(course:int):
    return 120

def exam_slots(course: int):
    # return int(exam_length(course)//15)
    return int(data.get_course_duration(course)//15)

exam_spans = {c: data.begin_end_pairs(exam_slots(c)) for c in range(data.n_courses)}

E = tt.addVars(
    [
        (c ,b ,e, d)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

C = tt.addVars(
    [
        
        (c,r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.INTEGER,
    lb=0,
    ub=230,
    name="C",
)



## Hard Constraints

In [3]:
def signum(x):
    if x>0:
        return 1
    return 0

def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in exam_spans[s] if  data.overlap(b,e,t)]

In [4]:
# Constraint 1: Each exam is scheduled exactly once
tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course"
)

tt.update()

In [5]:
# Constraint 2: Room capacity constraint C[c, r]*E[c, b, e, d] <= room_capacity[r] for all rooms

tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
                          for c in range(data.n_courses)
                          for b,e in covering_sessions(c, t)) <= data.get_room_capacity(r)
              for d in range(data.n_days)
              for r in range(data.n_rooms)
              for t in range(data.n_times)),
              name = "room_capacity_constraint"
             )
tt.update()


In [6]:
# Constraint 3: All students for a course should write the exam

tt.addConstrs(
    gp.quicksum(C[c,r] for r in range(data.n_rooms)) == data.get_course_strength(c)
    for c in range(data.n_courses)
)

tt.update()


In [7]:
# Constraint 4: Student clash constraint

tt.addConstrs((gp.quicksum(E[c, b, e, d] for c in data.intersection_pairs[i] for b, e in covering_sessions(c, t)) <= 1
               for i in range(data.n_intersection_pairs)
               for d in range(data.n_days)
               for t in range(data.n_times)),
              name='student_clash_constraint'
             )

tt.update()

In [8]:
# Constraint 5: No course can have more than 3 rooms alloted

M = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'M',
)

D1 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D1',
)

D2 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D2',
)

for c,r in M:
    room_capacity = data.get_room_capacity(r)
    tt.addConstr(M[c,r] <=1, name = 'M1_constraint')
    tt.addConstr(M[c,r] <=C[c,r], name = 'M2_constraint')
    tt.addConstr(M[c,r] >=D1[c,r], name = 'M3_constraint')
    tt.addConstr(M[c,r] >=C[c,r]-(room_capacity*(1-D2[c,r])), name = 'M4_constraint')
    tt.addConstr(D1[c,r]+D2[c,r] == 1, name = 'M5_constraint')

tt.addConstrs(
    (
        gp.quicksum(M[c, r] for r in range(data.n_rooms)) <= 3
        for c in range(data.n_courses)
    ),
    name="course_room_constraint",
)

tt.update()

In [9]:
# Constraint 6: Lab exams are conducted in labs

tt.addConstrs(
    (
        gp.quicksum(C[c, r] for r in (data.get_not_rooms_course(c)) )== 0
        for c in range(data.n_courses)
    ),
    name="lab_exam_constraint",
)

tt.update()

Course: 0
Rooms: [4, 5, 18]
Course: 1
Rooms: [4, 5, 18]
Course: 2
Rooms: [4, 5, 18]
Course: 3
Rooms: [4, 5, 18]
Course: 4
Rooms: [0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Course: 5
Rooms: [4, 5, 18]
Course: 6
Rooms: [4, 5, 18]
Course: 7
Rooms: [4, 5, 18]
Course: 8
Rooms: [4, 5, 18]
Course: 9
Rooms: [4, 5, 18]
Course: 10
Rooms: [4, 5, 18]
Course: 11
Rooms: [4, 5, 18]
Course: 12
Rooms: [4, 5, 18]
Course: 13
Rooms: [4, 5, 18]
Course: 14
Rooms: [4, 5, 18]
Course: 15
Rooms: [4, 5, 18]
Course: 16
Rooms: [4, 5, 18]
Course: 17
Rooms: [4, 5, 18]
Course: 18
Rooms: [4, 5, 18]
Course: 19
Rooms: [4, 5, 18]
Course: 20
Rooms: [4, 5, 18]
Course: 21
Rooms: [4, 5, 18]
Course: 22
Rooms: [4, 5, 18]
Course: 23
Rooms: [4, 5, 18]
Course: 24
Rooms: [4, 5, 18]
Course: 25
Rooms: [4, 5, 18]
Course: 26
Rooms: [4, 5, 18]
Course: 27
Rooms: [4, 5, 18]
Course: 28
Rooms: [4, 5, 18]
Course: 29
Rooms: [4, 5, 18]
Course: 30
Rooms: [0, 1, 2, 3, 6, 7, 8, 9,

In [10]:
# Constraint 7: No triplet courses should be scheduled in the day

tt.addConstrs(
    (
        (E.sum(c1, '*', '*', d) + E.sum(c2, '*', '*', d) + E.sum(c3, '*', '*', d) <= 2)
        for c1, c2, c3 in data.intersection_triplets
        for d in range(data.n_days)
    ),
    name="triplet_constraint",
)


{(16, 18, 37, 0): <gurobi.Constr *Awaiting Model Update*>,
 (16, 18, 37, 1): <gurobi.Constr *Awaiting Model Update*>,
 (16, 18, 37, 2): <gurobi.Constr *Awaiting Model Update*>,
 (16, 18, 37, 3): <gurobi.Constr *Awaiting Model Update*>,
 (16, 18, 37, 4): <gurobi.Constr *Awaiting Model Update*>,
 (16, 18, 37, 5): <gurobi.Constr *Awaiting Model Update*>,
 (18, 37, 51, 0): <gurobi.Constr *Awaiting Model Update*>,
 (18, 37, 51, 1): <gurobi.Constr *Awaiting Model Update*>,
 (18, 37, 51, 2): <gurobi.Constr *Awaiting Model Update*>,
 (18, 37, 51, 3): <gurobi.Constr *Awaiting Model Update*>,
 (18, 37, 51, 4): <gurobi.Constr *Awaiting Model Update*>,
 (18, 37, 51, 5): <gurobi.Constr *Awaiting Model Update*>,
 (16, 37, 50, 0): <gurobi.Constr *Awaiting Model Update*>,
 (16, 37, 50, 1): <gurobi.Constr *Awaiting Model Update*>,
 (16, 37, 50, 2): <gurobi.Constr *Awaiting Model Update*>,
 (16, 37, 50, 3): <gurobi.Constr *Awaiting Model Update*>,
 (16, 37, 50, 4): <gurobi.Constr *Awaiting Model Update*

## Soft Constraints

In [11]:
# # 1. No 2 exams on a day for a student

# # minimize summation courseintersection(c1, c2) * ((E[c1, *, *, d] + E[c2, *, *, d]) > 1) for all c1, c2, d



X = tt.addVars(
    [
        (c1, c2, d)
        for c1, c2 in data.intersection_pairs
        for d in range(data.n_days)
    ],
    vtype = GRB.BINARY, 
    name = 'X'
)

for c1, c2 in data.intersection_pairs:
    for d in range(data.n_days):
        tt.addConstr(E.sum(c1, '*', '*', d) + E.sum(c2, '*', '*', d) - X[c1, c2, d] <= 1 ,
                        name = 'soft_constraint1'
                    )

## Objective Function

In [12]:
# obj is summation of X[c1, c2, d] * weight(c1, c2) for all c1, c2, d
# for c1, c2 in data.intersection_pairs:
#     for d in range(data.n_days):
#         obj1 += X[c1, c2, d] 
obj1 = gp.quicksum(X[c1, c2, d]*(len(data.get_intersection_of_courses(c1, c2))) for c1, c2 in data.intersection_pairs for d in range(data.n_days))
for c1, c2 in data.intersection_pairs:
        print(len(data.get_intersection_of_courses(c1, c2)))
        print(data.get_course_title(c1), data.get_course_title(c2))
        print(data.course_student[c1], data.course_student[c2])
        print(data.get_intersection_of_courses(c1, c2))
        print('----------------------------------')


59
Data Structures and Algorithms Database Lab
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 377, 378, 402, 403, 404, 406, 407, 408, 410, 412, 413, 414, 439, 440, 444, 447, 448, 454, 456, 487, 549, 550, 551, 552, 553, 554, 555, 556, 557, 559, 560, 561, 562, 564, 566, 567]

In [13]:
# Soft Constraint 2: minimize the number of rooms used
obj2 = gp.quicksum(M[c,r] for c in range(data.n_courses) for r in range(data.n_rooms))



In [14]:
# obj3 = gp.quicksum(E[c, b, e, d] for c in range(data.n_courses) for b, e, d in data.get_preferred_slots(c))
# for c in range(data.n_courses):
#     for b, e, d in data.get_preferred_slots(c):
#         print("c: ", c, "b: ", b, "e: ", e, "d: ", d)
#         obj3 += E[c, b, e, d]

#add veriable Z[c] for each course c. Z[c] = 1 if the course is scheduled in the preferred slot
Z = tt.addVars(
    [
        (c)
        for c in range(data.n_courses)
    ],
    vtype=GRB.BINARY,
    name="Z",
)

tt.addConstrs(
    (Z[c] == gp.quicksum((E[c, b, e, d] for b, e, d in data.get_preferred_slots(c)))
    for c in range(data.n_courses)), 
    name="soft_constraint3"
)

tt.update()

obj3 = gp.quicksum(Z[c] for c in range(data.n_courses))

In [15]:
objective =  1*obj2 - 5*obj3 + 7*obj1
tt.setObjective(objective, sense=GRB.MINIMIZE)
tt.update()

In [16]:
# import time


# def cb(model, where):
#     if where == GRB.Callback.MIPNODE:
#         # Get model objective
#         obj = model.cbGet(GRB.Callback.MIPNODE_OBJBST)

#         # Has objective changed?
#         if abs(obj - model._cur_obj) > 1e-8:
#             # If so, update incumbent and time
#             model._cur_obj = obj
#             model._time = time.time()

#     # Terminate if objective has not improved in 60s
#     if time.time() - model._time > 120:
#         model.terminate()

tt._cur_obj = float('inf')
# tt._time = time.time()

# tt.optimize(callback=cb)
# # put time limit of 1200 seconds

# run it for 180 sec and dual gap 5 %

tt.setParam('MIPGap', 0.25)
tt.setParam('TimeLimit', 300)
tt.optimize()

Set parameter MIPGap to value 0.25
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.1 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  300
MIPGap  0.25

Academic license 2615578 - for non-commercial use only - registered to sa___@iiitb.ac.in
Optimize a model with 89730 rows, 15039 columns and 1030669 nonzeros
Model fingerprint: 0xfc621c5d
Model has 7068 quadratic constraints
Variable types: 0 continuous, 15039 integer (13272 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 3e+02]
  QRHS range       [6e+00, 2e+02]
Presolve removed 80126 rows and 4327 columns
Presolve time: 3.53s
Presolved: 170212 rows, 164528 column

In [17]:
from openpyxl import Workbook

def clash_report(X_sol, wb):
    wb.append(["Course 1", "Course 2", "Clash"])  
    for (c1, c2) in data.intersection_pairs:
        clash = 0
        for d in range(data.n_days):
            clash += X_sol[c1, c2, d]
        print(c1, c2, clash)
        wb.append([data.course_id_code[c1], data.course_id_code[c2], "True" if clash>=1 else "False"])
        

def exam_slot_report(Esol, Z_sol, wb):
    wb.append(["Course Code", "Title", "size", "CSE/ECE", "day", "duration", "begin", "end", "preferred_time_slot"])

    for c, b, e, d in Esol:
        wb.append([data.course_id_code[c], data.get_course_title(c), data.get_course_strength(c), data.get_dept(c), data.days[d], data.get_course_duration(c), data.times[b], data.times[e], "True" if Z_sol[c] else "False"])

def seating_report(C_sol, Csol, wb):
    wb.append(["Course Code", "Title", "Room", "Room Type", "size", "Room Number", "Capacity", "Seating"])

    for c, r in Csol:
        wb.append([data.course_id_code[c], data.get_course_title(c), data.get_room_num(r), data.get_course_category(c), data.get_course_strength(c), r, data.get_room_capacity(r), C_sol[c, r]])

def make_report(s):
    C_sol = s['C_sol']
    E_sol = s['E_sol']
    M_sol = s['M_sol']
    X_sol = s['X_sol']
    Z_sol = s['Z_sol']
    Esol = []
    for i in E_sol:
        if E_sol[i] > 0:
            Esol.append(i)
    Csol = []
    for i in C_sol:
        if C_sol[i] > 0:
            Csol.append(i)
    # Create the clash report workbook sheet
    wb = Workbook()
    default_sheet = wb.active
    wb.remove(default_sheet)
    clash_sheet = wb.create_sheet(title='Clash Report')
    exam_slot = wb.create_sheet(title='Exam Slot report')
    seating_sheet = wb.create_sheet(title='Seating Report')

    clash_report(X_sol, clash_sheet)
    exam_slot_report(Esol, Z_sol, exam_slot)
    seating_report(C_sol, Csol, seating_sheet)

    save_path = "/mnt/c/Users/priya/OneDrive/Desktop/timetabling/timetable_report.xlsx"
    wb.save(save_path)



In [18]:
# extract E C M X values for all best 3 solution counts and print them
import pandas as pd
def extract_values(model):
    E_sol = model.getAttr('x', E)
    C_sol = model.getAttr('x', C)
    M_sol = model.getAttr('x', M)
    X_sol = model.getAttr('x', X)
    Z_sol = model.getAttr('x', Z)
    return E_sol, C_sol, M_sol, X_sol, Z_sol
# make a 3 pd dataframe for the sol

# E_sol C_sol M_sol X_sol are sets, i want to crerate a dataframe with columns E_sol, C_sol, M_sol, X_sol
def get_bestmax3_solutions(model):
    for i in range(min(1, model.SolCount)):
        s = {}
        model.setParam('SolutionNumber', i)
        E_sol, C_sol, M_sol, X_sol, Z_sol = extract_values(model)
        s['E_sol'] = E_sol
        s['C_sol'] = C_sol
        s['M_sol'] = M_sol
        s['X_sol'] = X_sol
        s['Z_sol'] = Z_sol
        make_report(s)

get_bestmax3_solutions(tt)


15 30 0.0
7 26 0.0
40 41 0.0
48 54 1.0
11 23 0.0
41 42 0.0
10 36 0.0
10 45 0.0
2 41 0.0
10 54 0.0
3 24 0.0
3 33 0.0
3 42 0.0
14 51 0.0
36 48 0.0
47 48 0.0
7 21 0.0
17 53 0.0
9 49 0.0
29 36 0.0
48 49 0.0
6 43 0.0
29 54 0.0
10 31 0.0
10 49 0.0
22 23 0.0
22 32 1.0
14 37 1.0
55 56 0.0
5 55 0.0
29 31 0.0
2 22 0.0
25 33 0.0
22 27 0.0
3 23 0.0
13 55 0.0
16 51 0.0
28 34 0.0
17 34 0.0
36 47 1.0
9 30 0.0
9 48 0.0
21 22 0.0
6 33 0.0
10 30 0.0
2 26 0.0
24 32 0.0
35 41 0.0
14 18 0.0
16 37 0.0
17 38 0.0
28 38 0.0
21 26 0.0
39 43 0.0
2 3 0.0
24 27 1.0
3 4 0.0
51 53 0.0
1 34 0.0
16 50 0.0
5 22 0.0
9 20 0.0
9 29 0.0
9 47 0.0
6 32 0.0
2 7 0.0
20 43 0.0
12 48 0.0
4 44 0.0
16 18 0.0
1 38 0.0
35 40 0.0
5 8 0.0
17 19 0.0
38 51 0.0
17 28 0.0
5 26 0.0
5 44 0.0
27 41 0.0
50 52 0.0
8 55 0.0
20 29 0.0
39 42 0.0
23 25 0.0
23 43 0.0
4 39 0.0
1 15 1.0
24 26 0.0
32 39 0.0
5 12 0.0
15 53 0.0
9 28 0.0
8 32 0.0
42 43 0.0
11 46 0.0
12 20 0.0
12 29 0.0
4 25 0.0
12 47 0.0
4 43 0.0
1 19 0.0
34 53 0.0
1 28 0.0
5 7 0.0
46 54

In [19]:
E_sol, C_sol, M_sol, X_sol, Z_sol = extract_values(tt)

solution = []
for i in E_sol:
    if E_sol[i] > 0:
        solution.append(i)

In [20]:
solution_rooms=[]
for i in C_sol:
    if C_sol[i] > 0:
        solution_rooms.append(i)

In [21]:
! pip install tabulate
! pip install matplotlib

In [22]:
for c,b,e,d in solution:
    l = []
    for(r) in range(data.n_rooms):
        if C_sol[c,r] > 0:
            l.append((data.get_room_num(r), C_sol[c,r]))
    print(f"Course {data.course_id_code[c]} has {data.get_course_strength(c)} and is scheduled in slot {data.times[b]} to {data.times[e]} on day {data.days[d]} in room {l}")

Course AIM 101 has 59 and is scheduled in slot 10:45 to 12:45 on day Sat in room [('A303', 59.0)]
Course AIM 102 has 59 and is scheduled in slot 09:45 to 12:45 on day Tue in room [('R103', 59.0)]
Course AIM 704 has 20 and is scheduled in slot 14:15 to 17:15 on day Mon in room [('A304/A305', 20.0)]
Course AIM 821 has 21 and is scheduled in slot 14:45 to 17:45 on day Sat in room [('Ramanujan Basement 03 (C- Block)', 21.0)]
Course AIM 831 has 175 and is scheduled in slot 09:45 to 12:45 on day Fri in room [('R106 (LAB)', 60.0), ('R107/R108 (LAB)', 115.0)]
Course AIM 832 has 93 and is scheduled in slot 14:00 to 17:00 on day Sat in room [('R103', 93.0)]
Course AIM 843 has 35 and is scheduled in slot 14:00 to 15:00 on day Mon in room [('Ramanujan Basement 04 (D- Block)', 35.0)]
Course AIM 845 has 7 and is scheduled in slot 09:15 to 12:15 on day Fri in room [('R104', 7.0)]
Course AIM 846 has 26 and is scheduled in slot 09:45 to 12:45 on day Mon in room [('R101', 26.0)]
Course AMS 401 has 4 and

In [23]:
from collections import defaultdict
from tabulate import tabulate

schedule_by_day = defaultdict(list)

for c, b, e, d in solution:
    rooms = []
    for r in range(data.n_rooms):
        if C_sol[c, r] > 0:
            rooms.append(f"{data.get_room_num(r)} (Capacity: {C_sol[c, r]})")
    entry = {
        "Course": data.course_id_code[c],
        "Strength": data.get_course_strength(c),
        "Time": f"{data.times[b]} - {data.times[e]}",
        "Rooms": ", ".join(rooms),
        "start": data.times[b]
    }
    day = data.days[d]
    schedule_by_day[day].append(entry)

weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri","Sat"]

print("EXAM TIMETABLE")
print("=" * 50)
for day in weekdays:
    if day in schedule_by_day:
        sorted_entries = sorted(
            schedule_by_day[day],
            key=lambda entry: (entry["start"], entry["Strength"])
        )
        for entry in sorted_entries:
            del entry["start"]
        print(f"\n{day}")
        print("-" * 50)
        print(tabulate(sorted_entries, headers="keys", tablefmt="grid"))

EXAM TIMETABLE

Mon
--------------------------------------------------
+------------+------------+---------------+---------------------------------------------------+
| Course     |   Strength | Time          | Rooms                                             |
+============+============+===============+===================================================+
| CSE 754    |         25 | 09:15 - 12:15 | R102 (Capacity: 25.0)                             |
+------------+------------+---------------+---------------------------------------------------+
| DHS 201-A  |        158 | 09:30 - 12:30 | R203 (Capacity: 158.0)                            |
+------------+------------+---------------+---------------------------------------------------+
| AIM 846    |         26 | 09:45 - 12:45 | R101 (Capacity: 26.0)                             |
+------------+------------+---------------+---------------------------------------------------+
| VLS 804    |         31 | 09:45 - 12:45 | R105 (Capacity: 31.0)

In [24]:
## Verification of the solution

# 1 - No student should have two exams at the same time

def time_to_minutes(time_str):
    """Converts a HH:MM string to minutes since midnight."""
    hh, mm = map(int, time_str.split(":"))
    return hh * 60 + mm

# Build a dictionary that maps each course (c) to its one exam slot details.
# Assume each course appears only once in the solution.
course_slots = {}
for c, b, e, d in solution:
    # Save day and exam start/end times (converted to minutes) for each course
    day = data.days[d]
    start = time_to_minutes(data.times[b])
    end   = time_to_minutes(data.times[e])
    course_slots[c] = (day, start, end)

# Check for any student conflicts between every pair of distinct courses.
clash_found = False
course_indices = list(course_slots.keys())
for i in range(len(course_indices)):
    c1 = course_indices[i]
    day1, start1, end1 = course_slots[c1]
    for j in range(i + 1, len(course_indices)):
        c2 = course_indices[j]
        day2, start2, end2 = course_slots[c2]
        # Check if courses are on the same day and if their times overlap:
        if day1 == day2 and (start1 < end2 and start2 < end1):
            # Check intersection of enrolled students; get_intersection_of_courses returns set of common student IDs.
            common_students = set(data.course_student[c1]) & set(data.course_student[c2])
            if common_students:
                clash_found = True
                print(f"Clash found between Course {data.course_id_code[c1]} and Course {data.course_id_code[c2]}")
                print("Common student(s):", common_students)
                print(f"--> {data.course_id_code[c1]}: {data.times[b]} - {data.times[e]} ;",  # later you can format times properly
                      f"{data.course_id_code[c2]}: {data.times[b]} - {data.times[e]}\n")

if not clash_found:
    print("No student clashes found!")


# 2 - The total number of students writing an exam in a room should not exceed the room capacity


room_violations = False

# Process each room separately
for r in range(data.n_rooms):
    # Build a schedule per day for room r.
    room_schedule = defaultdict(list)
    for c, b, e, d in solution:
        allocated = C_sol[c, r]
        if allocated > 0:
            day = data.days[d]
            start = time_to_minutes(data.times[b])
            end = time_to_minutes(data.times[e])
            course_code = data.course_id_code[c]
            room_schedule[day].append((start, end, allocated, course_code))
    
    # For each day, process the exam intervals for room r.
    for day, intervals in room_schedule.items():
        events = []
        for start, end, alloc, course in intervals:
            # When exam starts, add the number of students;
            # when exam ends, subtract that number.
            events.append((start, alloc))
            events.append((end, -alloc))
        events.sort(key=lambda x: x[0])
        
        current_alloc = 0
        for time, change in events:
            current_alloc += change
            if current_alloc > data.get_room_capacity(r):
                room_violations = True
                print(f"Room capacity violation on {day} in room {data.get_room_num(r)}:")
                print(f"   Room capacity: {data.room_capacity[r]}, allocated: {current_alloc}")
                break  # Report once per day per room

if not room_violations:
    print("No room capacity violations found!")


No student clashes found!
No room capacity violations found!


In [25]:
for s in range (data.n_students):
    l=[]
    course_list=data.get_courses_student(s)
    for c,b,e,d in solution:
        if E_sol[c,b,e,d] == 1 and c in course_list:
            l.append((data.course_id_code[c], data.times[b], data.times[e], data.days[d]))
    
    print(f"Student {s+1} has exams for courses {l}")

Student 1 has exams for courses [('AIM 101', '10:45', '12:45', 'Sat'), ('AIM 102', '09:45', '12:45', 'Tue'), ('CSE 102-B', '16:00', '18:00', 'Tue'), ('CSE 102 P-B', '10:30', '13:00', 'Thu'), ('CSE 104-B', '14:45', '17:45', 'Fri'), ('DAS 101-B', '14:45', '17:45', 'Wed'), ('DAS 101P-B', '10:15', '12:45', 'Mon'), ('DHS 101B', '09:45', '12:45', 'Fri'), ('DHS 201-B', '15:00', '18:00', 'Mon')]
Student 2 has exams for courses [('AIM 101', '10:45', '12:45', 'Sat'), ('AIM 102', '09:45', '12:45', 'Tue'), ('CSE 102-B', '16:00', '18:00', 'Tue'), ('CSE 102 P-B', '10:30', '13:00', 'Thu'), ('CSE 104-B', '14:45', '17:45', 'Fri'), ('DAS 101-B', '14:45', '17:45', 'Wed'), ('DAS 101P-B', '10:15', '12:45', 'Mon'), ('DHS 101B', '09:45', '12:45', 'Fri'), ('DHS 201-B', '15:00', '18:00', 'Mon')]
Student 3 has exams for courses [('AIM 101', '10:45', '12:45', 'Sat'), ('AIM 102', '09:45', '12:45', 'Tue'), ('CSE 102-B', '16:00', '18:00', 'Tue'), ('CSE 102 P-B', '10:30', '13:00', 'Thu'), ('CSE 104-B', '14:45', '17:

In [26]:
for s in range(data.n_students):
    print(f"Student {s+1} has exam on: " , [data.days[d] for c,b,e,d in solution if E_sol[c,b,e,d] == 1 and c in data.get_courses_student(s)])

Student 1 has exam on:  ['Sat', 'Tue', 'Tue', 'Thu', 'Fri', 'Wed', 'Mon', 'Fri', 'Mon']
Student 2 has exam on:  ['Sat', 'Tue', 'Tue', 'Thu', 'Fri', 'Wed', 'Mon', 'Fri', 'Mon']
Student 3 has exam on:  ['Sat', 'Tue', 'Tue', 'Thu', 'Fri', 'Wed', 'Mon', 'Fri', 'Mon']
Student 4 has exam on:  ['Sat', 'Tue', 'Tue', 'Thu', 'Fri', 'Wed', 'Mon', 'Fri', 'Mon']
Student 5 has exam on:  ['Sat', 'Tue', 'Tue', 'Thu', 'Fri', 'Wed', 'Mon', 'Fri', 'Mon']
Student 6 has exam on:  ['Sat', 'Tue', 'Tue', 'Thu', 'Fri', 'Wed', 'Mon', 'Fri', 'Mon']
Student 7 has exam on:  ['Sat', 'Tue', 'Tue', 'Thu', 'Fri', 'Wed', 'Mon', 'Fri', 'Mon']
Student 8 has exam on:  ['Sat', 'Tue', 'Tue', 'Thu', 'Fri', 'Wed', 'Mon', 'Fri', 'Mon']
Student 9 has exam on:  ['Sat', 'Tue', 'Tue', 'Thu', 'Fri', 'Wed', 'Mon', 'Fri', 'Mon']
Student 10 has exam on:  ['Sat', 'Tue', 'Tue', 'Thu', 'Fri', 'Wed', 'Mon', 'Fri', 'Mon']
Student 11 has exam on:  ['Sat', 'Tue', 'Tue', 'Thu', 'Fri', 'Wed', 'Mon', 'Fri', 'Mon']
Student 12 has exam on:  ['Sat

In [27]:
# print how many courses have got the preferred slots
for c in range(data.n_courses):
    print(f"Course {data.course_id_code[c]} has got the preferred slots: ", [data.times[b] for b,e,d in data.get_preferred_slots(c) if E_sol[c,b,e,d] == 1])

Course AIM 101 has got the preferred slots:  ['10:45']
Course AIM 102 has got the preferred slots:  ['09:45']
Course AIM 704 has got the preferred slots:  ['14:15']
Course AIM 821 has got the preferred slots:  ['14:45']
Course AIM 831 has got the preferred slots:  ['09:45']
Course AIM 832 has got the preferred slots:  ['14:00']
Course AIM 843 has got the preferred slots:  ['14:00']
Course AIM 845 has got the preferred slots:  ['09:15']
Course AIM 846 has got the preferred slots:  ['09:45']
Course AMS 401 has got the preferred slots:  ['09:45']
Course AMS 402 has got the preferred slots:  ['14:00']
Course AMS 403 has got the preferred slots:  ['14:15']
Course AMS 404 has got the preferred slots:  ['10:45']
Course COM 605 has got the preferred slots:  ['15:00']
Course CSE 102-A has got the preferred slots:  ['14:45']
Course CSE 102-B has got the preferred slots:  ['16:00']
Course CSE 102 P-A has got the preferred slots:  ['09:45']
Course CSE 102 P-B has got the preferred slots:  ['10:30'

In [28]:
for c1, c2 in data.intersection_pairs:
    clash = 0
    for d in range(data.n_days):
        clash += X_sol[c1, c2, d]
        if(X_sol[c1, c2, d] > 0):
            print(data.course_id_code[c1], data.course_id_code[c2], d)
            
    print(data.course_id_code[c1], data.course_id_code[c2], clash>0)

CSE 102-B DAS 101P-B False
AIM 845 CSE 824 False
DHS 306 DHS 308 False
ECE 303P EGC 301 4
ECE 303P EGC 301 True
AMS 403 CSE 718 False
DHS 308 DHS 314 False
AMS 402 DHS 110 False
AMS 402 ECE 212 False
AIM 704 DHS 308 False
AMS 402 EGC 301 False
AIM 821 CSE 754 False
AIM 821 DAS 839 False
AIM 821 DHS 314 False
CSE 102-A EGC 121-B False
DHS 110 ECE 303P False
ECE 303 ECE 303P False
AIM 845 CSE 606 False
CSE 102 P-B EGC 123-B False
AMS 401 ECE 304 False
DAS 101-A DHS 110 False
ECE 303P ECE 304 False
AIM 843 DHS 315 False
DAS 101-A EGC 301 False
AMS 402 DAS 101P-A False
AMS 402 ECE 304 False
CSE 704 CSE 718 False
CSE 704 DAS 605 2
CSE 704 DAS 605 True
CSE 102-A DHS 201-A 0
CSE 102-A DHS 201-A True
VLS 603 VLS 804 False
AIM 832 VLS 603 False
DAS 101-A DAS 101P-A False
AIM 704 CSE 704 False
CSE 816 DAS 839 False
CSE 704 CSE 836 False
AIM 821 CSE 718 False
COM 605 VLS 603 False
CSE 102 P-A EGC 121-B False
DAS 101-B DHS 101B False
CSE 102 P-B DHS 101B False
DHS 110 ECE 303 3
DHS 110 ECE 303 Tru

In [29]:
# print all X(c1, c2, d) that are non zeros
for c1, c2 in data.intersection_pairs:
    for d in range(data.n_days):
        if X_sol[c1, c2, d] == 0:
            print(data.course_id_code[c1], data.course_id_code[c2], d)
        # print sum of E[c1, *, *, d] + E[c2, *, *, d] for that d
        Ec1d = 0
        for b, e in exam_spans[c1]:
            Ec1d += E_sol[c1, b, e, d]
        Ec2d = 0
        for b, e in exam_spans[c2]:
            Ec2d += E_sol[c2, b, e, d]
        print(data.course_id_code[c1], data.course_id_code[c2], d, Ec1d, Ec2d)
        print(X_sol[c1, c2, d])

CSE 102-B DAS 101P-B 0
CSE 102-B DAS 101P-B 0 0.0 1.0
0.0
CSE 102-B DAS 101P-B 1
CSE 102-B DAS 101P-B 1 1.0 0.0
-0.0
CSE 102-B DAS 101P-B 2
CSE 102-B DAS 101P-B 2 0.0 0.0
-0.0
CSE 102-B DAS 101P-B 3
CSE 102-B DAS 101P-B 3 0.0 0.0
-0.0
CSE 102-B DAS 101P-B 4
CSE 102-B DAS 101P-B 4 0.0 0.0
-0.0
CSE 102-B DAS 101P-B 5
CSE 102-B DAS 101P-B 5 0.0 0.0
-0.0
AIM 845 CSE 824 0
AIM 845 CSE 824 0 0.0 0.0
-0.0
AIM 845 CSE 824 1
AIM 845 CSE 824 1 0.0 0.0
-0.0
AIM 845 CSE 824 2
AIM 845 CSE 824 2 0.0 0.0
-0.0
AIM 845 CSE 824 3
AIM 845 CSE 824 3 0.0 1.0
-0.0
AIM 845 CSE 824 4
AIM 845 CSE 824 4 1.0 0.0
-0.0
AIM 845 CSE 824 5
AIM 845 CSE 824 5 0.0 0.0
0.0
DHS 306 DHS 308 0
DHS 306 DHS 308 0 0.0 0.0
-0.0
DHS 306 DHS 308 1
DHS 306 DHS 308 1 0.0 0.0
-0.0
DHS 306 DHS 308 2
DHS 306 DHS 308 2 0.0 1.0
-0.0
DHS 306 DHS 308 3
DHS 306 DHS 308 3 0.0 0.0
-0.0
DHS 306 DHS 308 4
DHS 306 DHS 308 4 0.0 0.0
-0.0
DHS 306 DHS 308 5
DHS 306 DHS 308 5 1.0 0.0
-0.0
ECE 303P EGC 301 0
ECE 303P EGC 301 0 0.0 0.0
-0.0
ECE 303P 

In [30]:
print(solution)

[(0, 11, 19, 5), (1, 7, 19, 1), (2, 22, 34, 0), (3, 24, 36, 5), (4, 7, 19, 4), (5, 21, 33, 5), (6, 21, 25, 0), (7, 5, 17, 4), (8, 7, 19, 0), (9, 7, 19, 1), (10, 21, 33, 1), (11, 22, 32, 1), (12, 11, 19, 1), (13, 25, 33, 5), (14, 24, 36, 0), (15, 29, 37, 1), (16, 7, 17, 1), (17, 10, 20, 3), (18, 5, 13, 3), (19, 24, 36, 4), (20, 11, 19, 5), (21, 21, 27, 5), (22, 21, 31, 2), (23, 21, 33, 0), (24, 5, 17, 0), (25, 11, 19, 1), (26, 21, 33, 3), (27, 22, 34, 0), (28, 24, 36, 2), (29, 21, 33, 0), (30, 9, 19, 0), (31, 10, 20, 2), (32, 11, 19, 2), (33, 29, 37, 4), (34, 7, 19, 4), (35, 21, 33, 3), (36, 0, 8, 3), (37, 6, 18, 0), (38, 25, 37, 0), (39, 10, 18, 0), (40, 10, 18, 5), (41, 5, 17, 2), (42, 26, 34, 4), (43, 10, 18, 1), (44, 5, 17, 2), (45, 25, 37, 5), (46, 7, 19, 0), (47, 25, 37, 3), (48, 7, 19, 4), (49, 7, 13, 2), (50, 7, 19, 5), (51, 25, 37, 5), (52, 11, 19, 2), (53, 25, 33, 2), (54, 21, 33, 4), (55, 7, 19, 1), (56, 7, 19, 0)]


In [31]:
count = 0
for c1, c2 in data.intersection_pairs:
    for d in range(data.n_days):
        se=0
        for b, e in exam_spans[c1]:
            se += E_sol[c1, b, e, d] 
        for b, e in exam_spans[c2]:
            se += E_sol[c2, b, e, d]
        if(se > 1):
            count+=1
            print(f'{len(data.get_intersection_of_courses(c1,c2))},X: {X_sol[c1, c2, d]}')

print(count)

61,X: 1.0
7,X: 1.0
158,X: 1.0
61,X: 1.0
1,X: 1.0
59,X: 1.0
3,X: 1.0
5,X: 1.0
1,X: 1.0
1,X: 1.0
1,X: 1.0
2,X: 1.0
123,X: 1.0
59,X: 1.0
3,X: 1.0
15


In [32]:
len(data.intersection_pairs)

329

In [ ]:
# print 